In [1]:
import pandas as pd
import numpy as np

import sklearn.linear_model
import statsmodels.api as sm
from scipy import stats
from scipy.stats import zscore

from collections import Counter

In [2]:
cancer_genes = pd.read_csv("../../COSMIC/Census_allTue Nov 10 13_38_57 2020.csv")
cancer_genes = cancer_genes["Gene Symbol"].tolist()

In [3]:
len(cancer_genes)

723

In [4]:
celllines_mapped = pd.read_csv("../MCF7_removal/Celllines_mapping_manual_noMCF7.csv")
celllines_mutations = pd.read_csv("../../GEMICCL/Mutation.csv", sep = "\t")

C:\Users\ds728\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
celllines_mapped

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,dsIdx,sCode,sName,dsCode,dsPert,dsAmount,dsUser1,...,ID,OI,OX,RX,ST,SX,SY,TAG,WW,Tissue
0,0,0,0,1,S404286,Batch5_SHP77_2,20140917162616309-936809,Batch5_SHP77,31.0,Batch5_SHP77_2,...,SHP-77,[],['NCBI_TaxID=9606; ! Homo sapiens'],"['PubMed=212181;', 'PubMed=3518877;', 'PubMed=...","['Source(s): ATCC; Cosmic-CLP; ECACC', 'Amelog...",Male,SHP77; Shadyside Hospital Pittsburgh-77,SHP77,['https://dtp.cancer.gov/discovery_development...,Lung
1,1,1,1,2,S404286,Batch5_SHP77_2,20140917162617937-936810,Batch5_SHP77,31.0,Batch5_SHP77_2,...,SHP-77,[],['NCBI_TaxID=9606; ! Homo sapiens'],"['PubMed=212181;', 'PubMed=3518877;', 'PubMed=...","['Source(s): ATCC; Cosmic-CLP; ECACC', 'Amelog...",Male,SHP77; Shadyside Hospital Pittsburgh-77,SHP77,['https://dtp.cancer.gov/discovery_development...,Lung
2,2,2,2,3,S404298,Batch5_PC9_2,20140917162619670-936811,Batch5_PC9,66.1,Batch5_PC9_2,...,PC-9,[],['NCBI_TaxID=9606; ! Homo sapiens'],"['DOI=10.5795/jjscc.24.451;', 'PubMed=1737336;...","['Source(s): PubMed=25877200', 'Amelogenin: X'...",Male,PC9; PC-9/S1; PC-9S1,PC9,['http://tcpaportal.org/mclp/'],Lung
3,3,3,3,4,S404298,Batch5_PC9_2,20140917162621386-936812,Batch5_PC9,66.1,Batch5_PC9_2,...,PC-9,[],['NCBI_TaxID=9606; ! Homo sapiens'],"['DOI=10.5795/jjscc.24.451;', 'PubMed=1737336;...","['Source(s): PubMed=25877200', 'Amelogenin: X'...",Male,PC9; PC-9/S1; PC-9S1,PC9,['http://tcpaportal.org/mclp/'],Lung
4,4,4,4,5,S404285,Batch5_SHP77_1,20140917162626101-936815,Batch5_SHP77,31.0,Batch5_SHP77_1,...,SHP-77,[],['NCBI_TaxID=9606; ! Homo sapiens'],"['PubMed=212181;', 'PubMed=3518877;', 'PubMed=...","['Source(s): ATCC; Cosmic-CLP; ECACC', 'Amelog...",Male,SHP77; Shadyside Hospital Pittsburgh-77,SHP77,['https://dtp.cancer.gov/discovery_development...,Lung
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1229,1229,1551,1551,1292,S404269,Batch7_EKVX_3,20140917165959026-938212,Batch7_EKVX,56.1,Batch7_EKVX_3,...,EKVX,[],['NCBI_TaxID=9606; ! Homo sapiens'],"['PubMed=2041050;', 'PubMed=3957463;', 'PubMed...",['Source(s): Cosmic-CLP; PubMed=19372543; PubM...,Male,NaN,EKVX,['https://dtp.cancer.gov/discovery_development...,Lung
1230,1230,1552,1552,1293,S404217,Batch6A_DU4475_3,20140917170000427-938213,Batch6A_DU4475,NaN,Batch6A_DU4475_3,...,DU4475,[],['NCBI_TaxID=9606; ! Homo sapiens'],"['PubMed=221108;', 'PubMed=10862037;', 'PubMed...",['Source(s): ATCC; Cosmic-CLP; DSMZ; PubMed=25...,Female,DU-4475; Du-4475; DU 4475; Du 4475; Duke Unive...,DU4475,['https://physics.cancer.gov/docs/bioresource/...,Breast
1231,1231,1553,1553,1294,S404217,Batch6A_DU4475_3,20140917170001804-938214,Batch6A_DU4475,NaN,Batch6A_DU4475_3,...,DU4475,[],['NCBI_TaxID=9606; ! Homo sapiens'],"['PubMed=221108;', 'PubMed=10862037;', 'PubMed...",['Source(s): ATCC; Cosmic-CLP; DSMZ; PubMed=25...,Female,DU-4475; Du-4475; DU 4475; Du 4475; Duke Unive...,DU4475,['https://physics.cancer.gov/docs/bioresource/...,Breast
1232,1232,1554,1554,1295,S404236,Batch6B_SF539_1,20140917170003120-938215,Batch6B_SF539,42.0,Batch6B_SF539_1,...,SF539,[],['NCBI_TaxID=9606; ! Homo sapiens'],"['PubMed=2041050;', 'PubMed=3019542;', 'PubMed...",['Source(s): Cosmic-CLP; PubMed=19372543; PubM...,Female,SF-539; SF-539 BT; SF 539,SF539,['https://dtp.cancer.gov/discovery_development...,Brain


In [6]:
# Get only cell lines with ID
celllines_mapped = celllines_mapped.drop_duplicates(subset = ["dsIdx"])
celllines_mapped = celllines_mapped.dropna(subset = ["ID"])
sample_data_tissue = celllines_mapped["DI"].tolist()

In [7]:
tissue_origin = pd.read_excel("../../Cellosaurus_data/Cellline_tissue_origin.xlsx", columns = ["Cancer", "Origin"])
tissue_origin_dict = tissue_origin.set_index("Cancer")["Origin"].to_dict()

In [8]:
tissue_type = []
for item in sample_data_tissue:
    if pd.isnull(item):
        tissue_type.append("Unknown") 
    elif item == "[]":
        tissue_type.append("Unknown")
    else:
        item = item.split(";")[2].split("'")[0].strip()
        tissue_type.append(item)
        
cell_origin = [tissue_origin_dict.get(item) for item in tissue_type]
cancer_subtype = [tissue_origin_dict.get(item) for item in tissue_type]
celllines_mapped["Tissue"] = cell_origin
celllines_mapped["Cancer_Type"] = tissue_type

In [9]:
Counter(cell_origin)

Counter({'Lung': 570,
         'Skin': 54,
         'Breast': 210,
         'Brain': 24,
         'Ovary': 38,
         'Blood': 30,
         'Colon': 34,
         'Kidney': 62,
         'Pancreas': 70,
         'Prostate': 18,
         'Bladder': 102,
         'Unknown': 6})

In [10]:
celllines_list = celllines_mapped["ID"].unique().tolist()

In [11]:
celllines_notinmutationset = set(celllines_list) - set(celllines_mutations['CellLineName_Cellosaurus'])
celllines_mapped_mutations = celllines_mapped[~celllines_mapped["ID"].isin(celllines_notinmutationset)]

In [12]:
tissue_labelled = pd.get_dummies(celllines_mapped_mutations, columns=["Tissue"])

In [13]:
metabolomics_data = pd.read_csv("../MCF7_removal/Metabolomics_data_noMCF7.csv")
metabolomics_data = metabolomics_data.set_index("ionIdx")

In [14]:
metabolomics_data

,ionMz,1,2,3,4,5,6,7,8,9,...,1287,1288,1289,1290,1291,1292,1293,1294,1295,1296
ionIdx,,,,,,,,,,,,,,,,,,,,,
1,57.0340,10755,9608,9434,9720,8356,7814,8998,9164,6230,...,6566,5004,6202,7946,5711,10677,6683,6125,5018,5627
2,58.0292,3507,3938,3836,4311,4534,7250,2338,3386,1506,...,1463,1281,2111,382,2691,1474,757,1059,2668,2061
3,59.0133,61335,64959,78616,78619,58873,64141,61454,62215,42346,...,33828,30939,41765,43246,33936,27913,22271,31315,67741,68680
4,59.0270,4178,4586,5914,4963,4268,5809,3971,6174,4275,...,4638,3525,3418,5450,4165,2648,7556,5147,5106,7744
5,59.0497,4124,6969,11827,11233,3640,4308,5750,6529,4072,...,2812,3423,1506,2841,2529,2642,2598,1084,3020,2301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,989.5599,2752,2993,1722,2304,2587,3685,6248,5451,3976,...,1362,1450,3704,3702,3075,3057,3320,3018,1607,1351
1096,990.7232,-286,-355,-348,167,-326,-357,557,814,683,...,677,979,1280,1354,524,182,1359,867,1535,1540
1097,992.1676,1574,1171,1039,576,1868,1414,2668,1465,2089,...,52,173,379,581,876,1130,181,288,204,164


In [15]:
cancer_genes_test = ["TP53"]

In [19]:
## We only have to calculate breast because we havent changed the rest of the data

In [20]:
for tissue in ["Breast"]:
    print("Analysing " + tissue)
    tissue_only = celllines_mapped[celllines_mapped["Tissue"] == tissue]
    if len(tissue_only["Cancer_Type"].unique()) > 1:
        tissue_labelled = pd.get_dummies(tissue_only, columns=["Cancer_Type"])
        analysistype = "labelled"
    elif len(tissue_only["Cancer_Type"].unique()) == 1:
        tissue_labelled = tissue_only
        analysistype = "unlabelled"

    count = 1
    gene_array = []
    for gene in cancer_genes:
        print("Calculating for " + gene + " %s/%s" % (str(count), str(len(cancer_genes))))
        gene_of_interest = celllines_mutations[celllines_mutations["HGNC"] == gene]
        gene_of_interest = gene_of_interest[gene_of_interest["MutationType_Raw"] != "Intron"]
        gene_of_interest = gene_of_interest[gene_of_interest["MutationType"] != "Silent"]
        gene_of_interest = gene_of_interest[gene_of_interest["MutationType"].notna()]
        gene_of_interest = gene_of_interest[gene_of_interest["MutationType"] != "Unknown"]
        gene_of_interest_celllines = set(gene_of_interest["CellLineName_Cellosaurus"])
            
        count+=1

            
        ## Get the cell lines with mutations
        cellines_with_mutation = tissue_labelled[tissue_labelled["ID"].isin(gene_of_interest_celllines)]["dsIdx"].tolist()
        cellines_without_mutation = tissue_labelled[~tissue_labelled["ID"].isin(gene_of_interest_celllines)]["dsIdx"].tolist()
        cellines_with_mutation = [str(x) for x in cellines_with_mutation]
        cellines_without_mutation = [str(x) for x in cellines_without_mutation]
            
         ## Get the metabolomics data
        mutated_celllines = metabolomics_data[cellines_with_mutation].transpose()
        nonmutated_celllines = metabolomics_data[cellines_without_mutation].transpose()
            
        # If there are no mutations found then ignore this gene
        if mutated_celllines.empty:
            print("No mutations found for " + gene + " in " + tissue)
            continue
        elif nonmutated_celllines.empty:
            print("No WT found for " + gene + " in " + tissue)
            continue
            ## Generate an array with the cell line metabolomics plus a tag for mutated for gene of interest
        mutated_celllines["Mutation"] = 1
        nonmutated_celllines["Mutation"] = 0
        
        celllines_labeled = pd.concat([mutated_celllines, nonmutated_celllines])
        celllines_labeled.index = celllines_labeled.index.map(str)
        
            ## Calculate tissue labels
        
        ################################################################ 
        ###### Run for data when there is only one tissue subtype ######
        ################################################################
        if analysistype == "unlabelled":
            X = (celllines_labeled.drop(["Mutation"], axis =1))
            Y = celllines_labeled["Mutation"]
            X= X.apply(zscore, axis =0)
    
            df_array = []
            Xsubset = X.copy()
            
            ## Generate a logistic regression model for every metabolite
            for item in range(1,X.shape[1]+1):
                Xsubset2 = Xsubset[item]
                X2 = sm.add_constant(Xsubset2)
                X2 = X2.join(tissue_labelled_tissue)
                est = sm.OLS(Y, X2)
                est2 = est.fit(disp=0)
                results_as_html = est2.summary().tables[1].as_html()
                summary = pd.read_html(results_as_html, header=0, index_col=0)[0]
                df_array.append(summary)
        
            ## Combine all of the resultant model data    
            array_concat = pd.concat(df_array)
            combined_array = array_concat[~array_concat.index.duplicated(keep='first')]
        #    combined_array = combined_array.iloc[combined_array["t"].abs().argsort()]
    
            ## Get only the T values, and then append them to the master array for concatenation later
            tvalue_only = pd.DataFrame(combined_array["t"])
            t_name = "T_" + gene
            tvalue_only.columns = ([t_name])
            tvalue_only = tvalue_only.drop(tissue_labels[1:])
    
            gene_array.append(tvalue_only)
            
            
        ################################################################ 
        ###### Run for data when there is only one tissue subtype ######
        ################################################################
        if analysistype == "labelled":    
            tissue_labels = []
            tissue_labels.append("dsIdx")
            for item in tissue_labelled.columns:
                if item.startswith("Cancer_Type"):
                    tissue_labels.append(item)
                    
        tissue_labelled_tissue = tissue_labelled[tissue_labels].set_index("dsIdx")
        tissue_labelled_tissue.index = tissue_labelled_tissue.index.rename("") 
        tissue_labelled_tissue.index = tissue_labelled_tissue.index.map(str)
        
        ## Add tissue labels to regression model
        celllines_labeled = celllines_labeled.join(tissue_labelled_tissue)
        
        ## Set up for the logistic regression -> X is all metabolomics, Y is the mutated vs non mutated
        X = (celllines_labeled.drop(["Mutation"] + tissue_labels[1:], axis =1))
        Y = celllines_labeled["Mutation"]
        ## Zscore the metabolites across the cell line axis
        X= X.apply(zscore, axis =0)
    
        df_array = []
        Xsubset = X.copy()
        
        ## Generate a logistic regression model for every metabolite
        for item in range(1,X.shape[1]+1):
            Xsubset2 = Xsubset[item]
            X2 = sm.add_constant(Xsubset2)
            X2 = X2.join(tissue_labelled_tissue)
            est = sm.OLS(Y, X2)
            est2 = est.fit(disp=0)
            results_as_html = est2.summary().tables[1].as_html()
            summary = pd.read_html(results_as_html, header=0, index_col=0)[0]
            df_array.append(summary)
        
        ## Combine all of the resultant model data    
        array_concat = pd.concat(df_array)
        combined_array = array_concat[~array_concat.index.duplicated(keep='first')]
    #    combined_array = combined_array.iloc[combined_array["t"].abs().argsort()]
    
        ## Get only the T values, and then append them to the master array for concatenation later
        tvalue_only = pd.DataFrame(combined_array["t"])
        t_name = "T_" + gene
        tvalue_only.columns = ([t_name])
        tvalue_only = tvalue_only.drop(tissue_labels[1:])
    
        gene_array.append(tvalue_only)
    allgenes_scores = pd.concat(gene_array, axis =1)
    allgenes_scores = allgenes_scores.drop("const", axis =0)
    filename = tissue + "_mutationspecific_metabolites_cancertypecontrolled.csv"
    allgenes_scores.to_csv(filename)

Analysing Breast
Calculating for A1CF 1/723
No mutations found for A1CF in Breast
Calculating for ABI1 2/723
No mutations found for ABI1 in Breast
Calculating for ABL1 3/723
Calculating for ABL2 4/723
Calculating for ACKR3 5/723
No mutations found for ACKR3 in Breast
Calculating for ACSL3 6/723
No mutations found for ACSL3 in Breast
Calculating for ACSL6 7/723
No mutations found for ACSL6 in Breast
Calculating for ACVR1 8/723
Calculating for ACVR2A 9/723
No mutations found for ACVR2A in Breast
Calculating for AFDN 10/723
No mutations found for AFDN in Breast
Calculating for AFF1 11/723
No mutations found for AFF1 in Breast
Calculating for AFF3 12/723
No mutations found for AFF3 in Breast
Calculating for AFF4 13/723
No mutations found for AFF4 in Breast
Calculating for AKAP9 14/723
Calculating for AKT1 15/723
No mutations found for AKT1 in Breast
Calculating for AKT2 16/723
No mutations found for AKT2 in Breast
Calculating for AKT3 17/723
No mutations found for AKT3 in Breast
Calculatin

Calculating for CREBBP 150/723
Calculating for CRLF2 151/723
No mutations found for CRLF2 in Breast
Calculating for CRNKL1 152/723
No mutations found for CRNKL1 in Breast
Calculating for CRTC1 153/723
No mutations found for CRTC1 in Breast
Calculating for CRTC3 154/723
Calculating for CSF1R 155/723
No mutations found for CSF1R in Breast
Calculating for CSF3R 156/723
No mutations found for CSF3R in Breast
Calculating for CSMD3 157/723
Calculating for CTCF 158/723
No mutations found for CTCF in Breast
Calculating for CTNNA2 159/723
No mutations found for CTNNA2 in Breast
Calculating for CTNNB1 160/723
No mutations found for CTNNB1 in Breast
Calculating for CTNND1 161/723
Calculating for CTNND2 162/723
Calculating for CUL3 163/723
No mutations found for CUL3 in Breast
Calculating for CUX1 164/723
No mutations found for CUX1 in Breast
Calculating for CXCR4 165/723
No mutations found for CXCR4 in Breast
Calculating for CYLD 166/723
Calculating for CYP2C8 167/723
No mutations found for CYP2C

No mutations found for HEY1 in Breast
Calculating for HIF1A 292/723
No mutations found for HIF1A in Breast
Calculating for HIP1 293/723
No mutations found for HIP1 in Breast
Calculating for HIST1H3B 294/723
No mutations found for HIST1H3B in Breast
Calculating for HIST1H4I 295/723
No mutations found for HIST1H4I in Breast
Calculating for HLA-A 296/723
Calculating for HLF 297/723
No mutations found for HLF in Breast
Calculating for HMGA1 298/723
No mutations found for HMGA1 in Breast
Calculating for HMGA2 299/723
No mutations found for HMGA2 in Breast
Calculating for HMGN2P46 300/723
No mutations found for HMGN2P46 in Breast
Calculating for HNF1A 301/723
Calculating for HNRNPA2B1 302/723
No mutations found for HNRNPA2B1 in Breast
Calculating for HOOK3 303/723
Calculating for HOXA11 304/723
No mutations found for HOXA11 in Breast
Calculating for HOXA13 305/723
No mutations found for HOXA13 in Breast
Calculating for HOXA9 306/723
No mutations found for HOXA9 in Breast
Calculating for HOXC

No mutations found for NCKIPSD in Breast
Calculating for NCOA1 438/723
Calculating for NCOA2 439/723
Calculating for NCOA4 440/723
Calculating for NCOR1 441/723
Calculating for NCOR2 442/723
Calculating for NDRG1 443/723
No mutations found for NDRG1 in Breast
Calculating for NF1 444/723
Calculating for NF2 445/723
Calculating for NFATC2 446/723
Calculating for NFE2L2 447/723
No mutations found for NFE2L2 in Breast
Calculating for NFIB 448/723
No mutations found for NFIB in Breast
Calculating for NFKB2 449/723
No mutations found for NFKB2 in Breast
Calculating for NFKBIE 450/723
No mutations found for NFKBIE in Breast
Calculating for NIN 451/723
No mutations found for NIN in Breast
Calculating for NKX2-1 452/723
No mutations found for NKX2-1 in Breast
Calculating for NONO 453/723
No mutations found for NONO in Breast
Calculating for NOTCH1 454/723
No mutations found for NOTCH1 in Breast
Calculating for NOTCH2 455/723
No mutations found for NOTCH2 in Breast
Calculating for NPM1 456/723
N

Calculating for SEPT9 588/723
No mutations found for SEPT9 in Breast
Calculating for SET 589/723
No mutations found for SET in Breast
Calculating for SETBP1 590/723
Calculating for SETD1B 591/723
No mutations found for SETD1B in Breast
Calculating for SETD2 592/723
Calculating for SETDB1 593/723
Calculating for SF3B1 594/723
No mutations found for SF3B1 in Breast
Calculating for SFPQ 595/723
No mutations found for SFPQ in Breast
Calculating for SFRP4 596/723
No mutations found for SFRP4 in Breast
Calculating for SGK1 597/723
No mutations found for SGK1 in Breast
Calculating for SH2B3 598/723
No mutations found for SH2B3 in Breast
Calculating for SH3GL1 599/723
No mutations found for SH3GL1 in Breast
Calculating for SHTN1 600/723
No mutations found for SHTN1 in Breast
Calculating for SIRPA 601/723
No mutations found for SIRPA in Breast
Calculating for SIX1 602/723
No mutations found for SIX1 in Breast
Calculating for SIX2 603/723
No mutations found for SIX2 in Breast
Calculating for SKI